In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt # to show graphical results
import re
import os
import tempfile

In [ ]:
# own modules
import modules.config as config
from modules.load_model import load_model
from modules.load_predict_data import load_predict_data
from prepare_images import prepare_images

In [ ]:
# get model name
MODEL_NAME = re.search(r"\/([A-Za-z0-9_-]{1,})\.zip", config.TRAINED_MODEL)
print("Current Model:", MODEL_NAME.group(1))

In [ ]:
# import pretrained model
model, label, tensorshape = load_model(config.TRAINED_MODEL)

In [ ]:
# show path to file(s) to be predicted
print("Prediction Source Path:", config.PREDICT_DATA)

In [ ]:
# If true, prediction pictures are being preprocessed (white background, 1:1 format, cutout)
# !! Contour Model needed, please see README.md !!
preprocessing = False

In [ ]:
# load data to be predicted
# predict_me -- contains the images to be predicted (as tf.stack)
# predict_file -- contains the images to be predicted (raw or preprocessed)
# predict_file_name -- name of the original file
predict_me = []
predict_file = []
predict_file_name = []

if preprocessing == True:
    with tempfile.TemporaryDirectory() as tmpdir:
        print(tmpdir)
        prepare_images(config.PREDICT_DATA, tmpdir)
        predict_me, predict_file, predict_file_name = load_predict_data(tmpdir, tensorshape[1], tensorshape[2])
else:
    predict_me, predict_file, predict_file_name = load_predict_data(config.PREDICT_DATA, tensorshape[1], tensorshape[2])
print("Pictures:", len(predict_me))

In [ ]:
# load pretrained model as probability model
prob_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])

In [ ]:
# make predictions
predictions = prob_model.predict(predict_me)

In [ ]:
# print the prediction results for each picture
for i in range(len(predict_file_name)):
    print("Filename:", predict_file_name[i])

    for key in label.keys():
        print(f"  - {key}: {predictions[i][label[key]]}")

    print("")

In [ ]:
# create export path if given
if config.PREDICT_RESULTS != "":
    if not os.path.exists(os.path.join(config.PREDICT_RESULTS,MODEL_NAME.group(1))):
        os.makedirs(os.path.join(config.PREDICT_RESULTS,MODEL_NAME.group(1)), exist_ok=True)

In [ ]:
# export the prediction results for each picture into a file
if config.PREDICT_RESULTS != "":
    with open(f"{config.PREDICT_RESULTS}/{MODEL_NAME.group(1)}/results.txt", "w") as file:
        for i in range(len(predict_file_name)):
            file.write(f"Filename: {predict_file_name[i]}\n")

            for key in label.keys():
                file.write(f"  - {key}: {predictions[i][label[key]]}\n")

            file.write("\n")

In [ ]:
for i in range(len(predict_me)):
    current_prediction = list(label.keys())[list(label.values()).index(predictions[i].argmax())]

    fig = plt.figure(figsize=(6, 8), facecolor="white")    
    fig.suptitle(f"Prediction {i+1}: {current_prediction}\n")
    fig.text(0.1, 0.93, f"Model: {MODEL_NAME.group(1)}")
    fig.text(0.1, 0.91, f"File: {predict_file_name[i]}")
    
    pic = fig.add_subplot(2,1,1)
    pic.axes.xaxis.set_ticks([])
    pic.axes.yaxis.set_ticks([])
    pic.imshow(predict_file[i])
    
    colrs = []
    for j in range(len(label)):
        if j == predictions[i].argmax():
            colrs.append("#DF2020")
        else:
            colrs.append("grey")
    
    hist = fig.add_subplot(2,1,2)
    hist.bar(list(label.keys()), predictions[i], color=colrs)
    hist.set_ylabel("Confidence")
    hist.set_ylim([0,1])

    if config.PREDICT_RESULTS != "":
        fig.savefig(f"{config.PREDICT_RESULTS}/{MODEL_NAME.group(1)}/pred_{predict_file_name[i]}_{current_prediction}.png", dpi=100, transparent=False, facecolor="w")
    else:
        fig.show()